**Alunos: Gabriel Rosa Galdino, Rodrigo Almeida Piropo**

**Matrícula: 202320098, 202320103**

# Análise e Implementação do Algoritmo de Dijkstra

## 1. Análise da Busca de Custo Uniforme (Algoritmo de Dijkstra)

A Busca de Custo Uniforme é uma estratégia de busca sem informação que pode ser implementada como uma busca de "Melhor Primeira Escolha", onde a fronteira (o conjunto de nós a serem explorados) é gerenciada por uma fila de prioridades ordenada pela função de custo do caminho acumulado ($g(n)$).

### **Propriedades do Algoritmo**

- **Completude:** **Sim**, o algoritmo é completo. Ele garante que encontrará uma solução se ela existir, desde que os custos das ações sejam maiores que um valor positivo mínimo ($\epsilon > 0$) para evitar ciclos de custo zero.
- **Entrega da Solução Ótima:** **Sim**, esta é a sua principal força. O algoritmo sempre encontra a solução de menor custo. Como ele expande os nós em ordem crescente de custo acumulado, a primeira vez que atinge um nó objetivo, é garantido que o caminho encontrado é o mais barato possível.
- **Complexidade de Tempo e Espaço:** A complexidade é de **$O(b^{1+\lfloor C^{*}/\epsilon\rfloor})$**, onde `b` é o fator de ramificação, $C^{*}$ é o custo da solução ótima e $\epsilon$ é o menor custo de ação. Isso demonstra que sua performance é sensível não apenas ao número de "passos", mas também ao custo total do caminho. Com implementações otimizadas (como heaps), a complexidade de tempo é frequentemente descrita como $O(E + V \log V)$, onde `V` são os vértices e `E` as arestas.


## 2. Comparativo: UCS (Dijkstra) vs. BFS vs. A*
Durante a prova oral citamos o A* por isso adicionamos na nossa análise 
| Característica | Busca de Custo Uniforme (UCS / Dijkstra) | Busca em Largura (BFS) | Busca A* (A-Star) |
| :--- | :--- | :--- | :--- |
| **Critério de Expansão** | Expande o nó com o menor **custo de caminho acumulado** ($g(n)$). | Expande o nó na **menor profundidade** (menor número de ações). | Expande o nó com o menor **custo estimado total** ($f(n) = g(n) + h(n)$). |
| **Tipo de Busca** | Sem informação (Não informada). | Sem informação (Não informada). | Com informação (Informada / Heurística). |
| **Estrutura de Dados**| Fila de prioridade (ordenada por $g(n)$). | Fila comum (FIFO). | Fila de prioridade (ordenada por $f(n)$). |
| **Garantia de Otimização** | **Sempre** encontra a solução de menor custo (para custos $\ge 0$). | Só garante a solução de menor custo se **todas as ações tiverem custos idênticos**. | **Sempre**, se a heurística $h(n)$ for **admissível** (nunca superestimar o custo real). |
| **Relação Fundamental** | É uma generalização da BFS. Pode ser visto como um caso especial de A* onde a heurística $h(n) = 0$. | É um caso especial da UCS, onde todos os custos de ação são iguais a 1. | É uma generalização da UCS (Dijkstra), adicionando uma heurística para guiar a busca. |

## 3. Implementação com Heap de Fibonacci

A eficiência do Algoritmo de Dijkstra depende fortemente da estrutura de dados usada para a fila de prioridade. Um **Heap de Fibonacci** é uma estrutura de dados que oferece um desempenho amortizado mais rápido para a operação de "diminuir a chave" (essencial quando encontramos um caminho mais curto para um nó que já está na fronteira). Isso torna a implementação geral mais rápida, especialmente em grafos densos.


### Algoritmo: UCS (Dijkstra)

In [8]:
import math
import heapq

def dijkstra_fibonacci_heap(graph, start_node, end_node):
    """
    Implementação prática do Dijkstra usando heapq (min-heap binário).
    A complexidade é O((V + E) log V), similar à versão com Fibonacci Heap
    para grafos de tamanho moderado.
    """
    
    distances = {node: math.inf for node in graph}
    distances[start_node] = 0
    previous_nodes = {node: None for node in graph}
    pq = [(0, start_node)]  # (distância, nó)
    nodes_explored = 0    
    
    while pq:
        nodes_explored += 1 
        current_dist, current_node = heapq.heappop(pq)
        
        if current_node == end_node: 
             break 

        if current_dist > distances[current_node]:
            continue
        
        for neighbor, weight in graph[current_node].items():
            new_dist = current_dist + weight
            
            if new_dist < distances[neighbor]:
                distances[neighbor] = new_dist
                previous_nodes[neighbor] = current_node
                heapq.heappush(pq, (new_dist, neighbor))
    
    return distances, previous_nodes, nodes_explored


def reconstruct_path(previous_nodes, start_node, end_node):
    path = []
    current = end_node
    while current is not None:
        path.insert(0, current)
        current = previous_nodes.get(current)
    return path if path and path[0] == start_node else None

### Algoritmo: Busca em Largura (BFS)

In [9]:
from collections import deque

def bfs(graph, start_node, end_node):
    """
    Implementação da Busca em Largura (BFS).
    Encontra o caminho com o menor número de passos.
    """
    # Fila armazena (nó_atual, caminho_até_aqui)
    queue = deque([(start_node, [start_node])])
    visited = {start_node}
    nodes_explored = 0
    
    while queue:
        nodes_explored += 1
        current_node, path = queue.popleft()
        
        if current_node == end_node:
            # Encontrou! Agora calcula o custo (distância) desse caminho
            cost = 0
            for i in range(len(path) - 1):
                cost += graph[path[i]][path[i+1]]
            return path, cost, nodes_explored
        
        for neighbor in graph[current_node]:
            if neighbor not in visited:
                visited.add(neighbor)
                new_path = list(path)
                new_path.append(neighbor)
                queue.append((neighbor, new_path))
                
    return None, 0, nodes_explored

### Algoritmo: A* (A-Star)

In [10]:
# Heurística: Distância em Linha Reta (SLD) até Bucharest
# (Necessária para o A*)
romania_sld_to_bucharest = {
    'Arad': 366, 'Bucharest': 0, 'Craiova': 160, 'Drobeta': 242,
    'Eforie': 161, 'Fagaras': 178, 'Giurgiu': 77, 'Hirsova': 151,
    'Iasi': 226, 'Lugoj': 244, 'Mehadia': 241, 'Neamt': 234,
    'Oradea': 380, 'Pitesti': 100, 
    'Rimnicu Vilcea': 193, 'Sibiu': 253, 'Timisoara': 329,
    'Urziceni': 80, 'Vaslui': 199, 'Zerind': 374
}

In [11]:
def a_star(graph, start_node, end_node, heuristic):
    """
    Implementação da Busca A* (A-Star).
    Usa g(n) (custo real) + h(n) (heurística) para guiar a busca.
    """
    
    # Fila de prioridade armazena: (f_cost, g_cost, nó_atual, caminho_até_aqui)
    # f_cost = g_cost + h_cost
    g_cost = 0
    h_cost = heuristic[start_node]
    f_cost = g_cost + h_cost
    
    pq = [(f_cost, g_cost, start_node, [start_node])]
    visited = {start_node: 0} # Armazena o g_cost mais baixo encontrado
    nodes_explored = 0
    
    while pq:
        nodes_explored += 1
        current_f, current_g, current_node, path = heapq.heappop(pq)
        
        if current_node == end_node:
            return path, current_g, nodes_explored # Retorna o caminho e o custo real (g)

        # Otimização: Se já encontramos um caminho melhor para este nó, ignora
        if current_g > visited.get(current_node, math.inf):
            continue
            
        for neighbor, weight in graph[current_node].items():
            new_g = current_g + weight
            
            # Se é um nó novo ou encontramos um caminho mais curto para ele
            if neighbor not in visited or new_g < visited.get(neighbor, math.inf):
                visited[neighbor] = new_g
                new_h = heuristic[neighbor]
                new_f = new_g + new_h
                heapq.heappush(pq, (new_f, new_g, neighbor, path + [neighbor]))

    return None, 0, nodes_explored

### Bloco de Execução e Comparação

In [12]:
romania_graph = {
    'Arad': {'Zerind': 75, 'Sibiu': 140, 'Timisoara': 118},
    'Zerind': {'Arad': 75, 'Oradea': 71},
    'Oradea': {'Zerind': 71, 'Sibiu': 151},
    'Sibiu': {'Arad': 140, 'Oradea': 151, 'Fagaras': 99, 'Rimnicu Vilcea': 80},
    'Timisoara': {'Arad': 118, 'Lugoj': 111},
    'Lugoj': {'Timisoara': 111, 'Mehadia': 70},
    'Mehadia': {'Lugoj': 70, 'Drobeta': 75},
    'Drobeta': {'Mehadia': 75, 'Craiova': 120},
    'Craiova': {'Drobeta': 120, 'Rimnicu Vilcea': 146, 'Pitesti': 138},
    'Rimnicu Vilcea': {'Sibiu': 80, 'Craiova': 146, 'Pitesti': 97},
    'Fagaras': {'Sibiu': 99, 'Bucharest': 211},
    'Pitesti': {'Rimnicu Vilcea': 97, 'Craiova': 138, 'Bucharest': 101},
    'Bucharest': {'Fagaras': 211, 'Pitesti': 101, 'Giurgiu': 90, 'Urziceni': 85},
    'Giurgiu': {'Bucharest': 90},
    'Urziceni': {'Bucharest': 85, 'Hirsova': 98, 'Vaslui': 142},
    'Hirsova': {'Urziceni': 98, 'Eforie': 86},
    'Eforie': {'Hirsova': 86},
    'Vaslui': {'Urziceni': 142, 'Iasi': 92},
    'Iasi': {'Vaslui': 92, 'Neamt': 87},
    'Neamt': {'Iasi': 87}
}

start = 'Arad'
end = 'Bucharest'

print(f"Comparando algoritmos de busca de '{start}' para '{end}':\n")

# --- BFS ---
path_bfs, cost_bfs, explored_bfs = bfs(romania_graph, start, end)
print("--- 1. Busca em Largura (BFS) ---")
if path_bfs:
    print(f"  Objetivo: Encontrar o menor NÚMERO DE PASSOS.")
    print(f"  Nós explorados: {explored_bfs}")
    print(f"  Passos: {len(path_bfs) - 1}")
    print(f"  Custo (Distância) do caminho: {cost_bfs}")
    print(f"  Caminho: {' -> '.join(path_bfs)}\n")
else:
    print("  Caminho não encontrado.\n")

# --- UCS (Dijkstra) ---
distances_ucs, prev_ucs, explored_ucs = dijkstra_fibonacci_heap(romania_graph, start, end)
path_ucs = reconstruct_path(prev_ucs, start, end)
cost_ucs = distances_ucs.get(end, math.inf)

print("--- 2. Busca de Custo Uniforme (UCS / Dijkstra) ---")
if path_ucs:
    print(f"  Objetivo: Encontrar o menor CUSTO TOTAL (Distância).")
    print(f"  Nós explorados: {explored_ucs}")
    print(f"  Passos: {len(path_ucs) - 1}")
    print(f"  Custo (Distância) do caminho: {cost_ucs}")
    print(f"  Caminho: {' -> '.join(path_ucs)}\n")
else:
    print("  Caminho não encontrado.\n")

# --- A* (A-Star) ---
path_astar, cost_astar, explored_astar = a_star(romania_graph, start, end, romania_sld_to_bucharest)
print("--- 3. Busca A* (A-Star) ---")
if path_astar:
    print(f"  Objetivo: Encontrar o menor CUSTO TOTAL (usando heurística).")
    print(f"  Nós explorados: {explored_astar}")
    print(f"  Passos: {len(path_astar) - 1}")
    print(f"  Custo (Distância) do caminho: {cost_astar}")
    print(f"  Caminho: {' -> '.join(path_astar)}\n")
else:
    print("  Caminho não encontrado.\n")

Comparando algoritmos de busca de 'Arad' para 'Bucharest':

--- 1. Busca em Largura (BFS) ---
  Objetivo: Encontrar o menor NÚMERO DE PASSOS.
  Nós explorados: 9
  Passos: 3
  Custo (Distância) do caminho: 450
  Caminho: Arad -> Sibiu -> Fagaras -> Bucharest

--- 2. Busca de Custo Uniforme (UCS / Dijkstra) ---
  Objetivo: Encontrar o menor CUSTO TOTAL (Distância).
  Nós explorados: 13
  Passos: 4
  Custo (Distância) do caminho: 418
  Caminho: Arad -> Sibiu -> Rimnicu Vilcea -> Pitesti -> Bucharest

--- 3. Busca A* (A-Star) ---
  Objetivo: Encontrar o menor CUSTO TOTAL (usando heurística).
  Nós explorados: 6
  Passos: 4
  Custo (Distância) do caminho: 418
  Caminho: Arad -> Sibiu -> Rimnicu Vilcea -> Pitesti -> Bucharest



A execução prática dos três algoritmos no grafo da Romênia validou as propriedades teóricas de cada um:

1.  **Busca em Largura (BFS):** Cumpriu seu objetivo de encontrar o caminho com o **menor número de passos** (3 passos). No entanto, esse caminho teve um custo (450) muito superior ao ótimo. Isso confirma que a BFS não garante a otimização de custo, a menos que todos os custos de ação sejam idênticos.

2.  **Busca de Custo Uniforme (UCS / Dijkstra):** Encontrou o **caminho de menor custo real** (418), confirmando sua principal força: a garantia de otimização. Por ser uma busca "cega", que se expande apenas com base no custo acumulado ($g(n)$), ela foi menos eficiente, precisando explorar **13 nós**.

3.  **Busca A* (A-Star):** Encontrou o **mesmo caminho ótimo** (418) que o Dijkstra, mas provou ser muito mais eficiente. Ao usar uma heurística (busca informada) para "guiar" sua expansão, o A* precisou explorar apenas **6 nós** (menos da metade que o Dijkstra) para chegar à mesma solução.

**Veredito:** O experimento mostra que, para problemas de caminho de menor custo, o **Dijkstra (UCS) é superior ao BFS**. No entanto, o **A* é superior ao Dijkstra (UCS) em eficiência**, pois encontra a mesma solução ótima com um custo computacional (nós explorados) significativamente menor.

## 4. O Problema das 8 Rainhas

O Algoritmo de Dijkstra (Busca de Custo Uniforme) não é a escolha ideal para resolver o Problema das 8 Rainhas. Na prática, ele se comportaria de forma muito ineficiente pela seguinte razão:

1.  **Tipo de Problema:** O Dijkstra (UCS) é um algoritmo otimizado para encontrar o **caminho de menor custo** (ex: a rota mais curta entre duas cidades). O problema das 8 Rainhas, por outro lado, é um **Problema de Satisfação de Restrições (CSP)**. O objetivo não é encontrar um "caminho barato", mas sim um "estado final" (um tabuleiro) que satisfaça um conjunto de regras (nenhuma rainha ataca a outra).

2.  **Custo do Caminho ($g(n)$):** Para forçar o Dijkstra a resolver isso, seria preciso definir um "custo" para cada passo (colocar uma rainha). O custo mais lógico seria `custo = 1` para cada rainha colocada.

3.  **A Consequência:** Se todo passo tem custo 1, o "custo total do caminho" ($g(n)$) se torna exatamente igual à profundidade da busca (o número de rainhas no tabuleiro).

4.  **Conclusão:** Quando o custo do caminho é igual à profundidade, a Busca de Custo Uniforme se torna **idêntica à Busca em Largura (BFS)**. O algoritmo exploraria *todos* os tabuleiros válidos com 1 rainha, depois *todos* com 2 rainhas, e assim por diante. Isso é computacionalmente inviável e uma das piores formas de resolver o problema.

O algoritmo correto para esse tipo de problema é o **Backtracking** (que é uma forma de Busca em Profundidade - DFS).

### Abaixo está a implementação do problema das 8 rainhas usando Dijkstra

In [13]:
import heapq

def solve_queens_with_dijkstra_ucs(n=8):
    """
    Resolve o problema das N-Rainhas usando a lógica de Busca de Custo Uniforme (Dijkstra).
    Isso é funcionalmente idêntico a uma Busca em Largura (BFS) e é muito ineficiente.
    """
    
    # --- Função auxiliar para verificar a segurança ---
    def is_safe_dijkstra(state):
        """
        Verifica se o último estado (rainha recém-colocada) é seguro.
        O 'state' é um tupla, ex: (1, 3, 0)
        """
        if len(state) <= 1:
            return True
        
        last_col = len(state) - 1
        last_row = state[last_col]
        
        # Checa as colunas anteriores
        for col in range(last_col):
            row = state[col]
            
            # Checa a linha
            if row == last_row:
                return False
            
            # Checa as diagonais
            if abs(row - last_row) == abs(col - last_col):
                return False
        
        return True

    # --- Lógica principal do Dijkstra (UCS) ---
    
    # A fila de prioridade armazena (custo_acumulado, estado_do_tabuleiro)
    # O 'estado' é uma tupla onde o índice é a coluna e o valor é a linha
    # Ex: (1, 3, 0) -> Rainha na (0,1), (1,3), (2,0)
    
    # Começamos com um custo 0 e um tabuleiro vazio ()
    pq = [(0, ())] 
    
    solutions = []
    
    while pq:
        # Pega o nó com o menor custo (g(n))
        current_cost, current_state = heapq.heappop(pq)
        
        current_col = len(current_state) # Coluna que vamos tentar preencher
        
        # --- Verificação de Objetivo ---
        if current_col == n:
            # Se o estado tem N rainhas, é uma solução completa
            solutions.append(current_state)
            continue # Continua buscando outras soluções

        # --- Geração de Vizinhos (Próximos Estados Válidos) ---
        
        # Para cada linha na coluna atual...
        for row_to_try in range(n):
            
            # Cria um novo estado potencial
            new_state = current_state + (row_to_try,)
            
            # Verifica se esse novo estado é válido
            if is_safe_dijkstra(new_state):
                
                # O custo para chegar a este novo estado é o custo antigo + 1
                new_cost = current_cost + 1
                
                # Adiciona o novo estado válido na fila de prioridade
                heapq.heappush(pq, (new_cost, new_state))
                
    return solutions

In [ ]:
solutions = solve_queens_with_dijkstra_ucs(n=8)

print(f"Total de {len(solutions)} soluções encontradas.")

# Imprimir as primeiras 3 soluções
for i, sol in enumerate(solutions[:3]):
    print(f"\n--- Solução {i+1} ---")
    
    # Coordenadas (Coluna, Linha)
    print("Posições (Coluna, Linha):")
    coord_list = []
    for col, row in enumerate(sol):
        coord_list.append(f"({col}, {row})")
    print("  " + ", ".join(coord_list))
    
    print("Tabuleiro:")
    board = [["." for _ in range(8)] for _ in range(8)]
    for col, row in enumerate(sol):
        board[row][col] = "Q"
    
    for row_data in board:
        print("  " + " ".join(row_data))

Total de 92 soluções encontradas.

--- Solução 1 ---
Posições (Coluna, Linha):
  (0, 0), (1, 4), (2, 7), (3, 5), (4, 2), (5, 6), (6, 1), (7, 3)
Tabuleiro:
  Q . . . . . . .
  . . . . . . Q .
  . . . . Q . . .
  . . . . . . . Q
  . Q . . . . . .
  . . . Q . . . .
  . . . . . Q . .
  . . Q . . . . .

--- Solução 2 ---
Posições (Coluna, Linha):
  (0, 0), (1, 5), (2, 7), (3, 2), (4, 6), (5, 3), (6, 1), (7, 4)
Tabuleiro:
  Q . . . . . . .
  . . . . . . Q .
  . . . Q . . . .
  . . . . . Q . .
  . . . . . . . Q
  . Q . . . . . .
  . . . . Q . . .
  . . Q . . . . .

--- Solução 3 ---
Posições (Coluna, Linha):
  (0, 0), (1, 6), (2, 3), (3, 5), (4, 7), (5, 1), (6, 4), (7, 2)
Tabuleiro:
  Q . . . . . . .
  . . . . . Q . .
  . . . . . . . Q
  . . Q . . . . .
  . . . . . . Q .
  . . . Q . . . .
  . Q . . . . . .
  . . . . Q . . .
